In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler


In [0]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("PySpark Example") \
    .getOrCreate()


In [0]:
# Sample data in a list format
data = [
    ("Alice", "Female", 29, 3000),
    ("Bob", "Male", 34, 4000),
    ("Charlie", "Female", 25, 3500),
    ("David", "Male", 40, 4500),
    ("Eve", "Female", 30, 3200)
]

# Define column names
columns = ["Name", "Gender", "Age", "Salary"]

# Create a DataFrame
df = spark.createDataFrame(data, columns)

# Show the original DataFrame
print("Original Data:")
df.show()



Original Data:
+-------+------+---+------+
|   Name|Gender|Age|Salary|
+-------+------+---+------+
|  Alice|Female| 29|  3000|
|    Bob|  Male| 34|  4000|
|Charlie|Female| 25|  3500|
|  David|  Male| 40|  4500|
|    Eve|Female| 30|  3200|
+-------+------+---+------+



In [0]:
# Step 1: Index the "Gender" column (convert categorical data to numerical)
indexer = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
df_indexed = indexer.fit(df).transform(df)

print("After StringIndexer (Gender -> GenderIndex):")
df_indexed.show()

After StringIndexer (Gender -> GenderIndex):
+-------+------+---+------+-----------+
|   Name|Gender|Age|Salary|GenderIndex|
+-------+------+---+------+-----------+
|  Alice|Female| 29|  3000|        0.0|
|    Bob|  Male| 34|  4000|        1.0|
|Charlie|Female| 25|  3500|        0.0|
|  David|  Male| 40|  4500|        1.0|
|    Eve|Female| 30|  3200|        0.0|
+-------+------+---+------+-----------+



In [0]:
# Step 2: One-hot encode the "GenderIndex" column
encoder = OneHotEncoder(inputCol="GenderIndex", outputCol="GenderVec")
df_encoded = encoder.fit(df_indexed).transform(df_indexed)

print("After OneHotEncoder (GenderIndex -> GenderVec):")
df_encoded.show()

com.databricks.backend.daemon.driver.DriverClientDestroyedException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$4(DriverClient.scala:755)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$2(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:273)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:269)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.threading.ContextBoundRunnable.withAttributionContext(ContextBoundRunnable.scala:6)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$1(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:73)
	at com.databricks.threading.ContextBoundRunnable.run(ContextBoundRunnable.scala:14)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:477)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)

In [0]:
# Step 3: Assemble the features (Age, Salary, and GenderVec) into a single feature vector
assembler = VectorAssembler(
    inputCols=["Age", "Salary", "GenderVec"],  # Input features
    outputCol="features"  # Output column with assembled vector
)

df_final = assembler.transform(df_encoded)

print("After VectorAssembler (Assembled Features):")
df_final.select("Name", "features").show(truncate=False)

# Stop the Spark session
spark.stop()